In [ ]:

class DataGenerator(keras.utils.Sequence):
    'Generates data for Keras'
    def __init__(self, Image_batch,Question_batch,Answer_label_batch, batch_size, dim=(224,224),
                 n_classes=1260, path="",shuffle=True):
        'Initialization'
        self.dim = dim
        self.batch_size = batch_size
        self.labels = labels
        self.list_IDs = list_IDs
        #self.n_channels = n_channels
        self.indexes = np.arange(len(y))
        self.n_classes = n_classes
        self.shuffle = shuffle
        
        self.Image_batch = Image_batch
        self.Question_batch = Question_batch
        self.Answer_label_batch = Answer_label_batch
        
        self.on_epoch_end()

    def __len__(self):
        'Denotes the number of batches per epoch'
        return len(self.indexes) // self.batch_size

    def __getitem__(self, index):
        Qstn = self.Question_batch[index * self.batch_size:(index + 1) * self.batch_size]
        IMG = self.Image_batch[index * self.batch_size:(index + 1) * self.batch_size]
        Ans = self.Answer_label_batch[index * self.batch_size:(index + 1) * self.batch_size]

        qstn = np.asarray([self.__get_question_(que) for que in Qstn])
        img = np.asarray([self.__get_Image_(colab_path+path) for path in IMG])
        ans = np.asarray([self.__get_output(c) for c in Ans])
       
        return tuple([img, qstn]), ans

    def __get_question_(self, que):
    
        #que_arr = (pad_sequences(t.texts_to_sequences([X_que]), maxlen=22, padding='post'))[0]
        sequences = tokenizer.texts_to_sequences(que)
        sequence_padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
        return sequence_padded

    def __get_Image_(self, path):

        img = cv2.imread(path)
        #resize image as (224,224,3)
        resized_img = cv.resize(img,self.dim)
        #normalize inage
        img = np.array(resized_img)/255.0
            
        return img

    def __get_output(self, answer):
        return tf.keras.utils.to_categorical(answer, num_classes=self.n_classes)

    def on_epoch_end(self):
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)


In [ ]:
class CustomDataGen_aug(tf.keras.utils.Sequence):
    
    def __init__(self, X_que, X_img, y,
                 batch_size,
                 shuffle=True):
        
        self.X_que = X_que
        self.X_img = X_img
        self.y = y
        self.batch_size = batch_size
        self.shuffle = shuffle
        self.indexes = np.arange(len(y))
        self.vocab_size = 10000
        self.embedding_dim = 256
        self.max_length = 22
        self.trunc_type='post'
        self.padding_type='post'
        self.oov_tok = "<OOV>"
        tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_tok)
        tokenizer.fit_on_texts(question_list_train)

    def on_epoch_end(self):
        if self.shuffle:
            self.indexes = np.random.permutation(self.indexes)
    
    def __get_input1(self, que):
    
        #que_arr = (pad_sequences(t.texts_to_sequences([X_que]), maxlen=22, padding='post'))[0]
        training_sequences = tokenizer.texts_to_sequences(que)
        training_padded = pad_sequences(training_sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
        return training_padded

    def __get_input2(self, path):

        img = cv2.imread(path)
        # a = np.random.uniform()
        # if a<0.25:
        #     img = aug1.augment_image(img)
        # elif a<0.5:
        #     img = aug2.augment_image(img)
        # elif a<0.75:
        #     img = aug3.augment_image(img)
        # else:
        #     img = img
        img = cv2.resize()    
        img = np.array(img)/255.0
            
        return img
    
    def __get_output(self, label):
        return tf.keras.utils.to_categorical(label, num_classes=1000)
    
    def __getitem__(self, index):

        batch_x0 = self.X_que[index * self.batch_size:(index + 1) * self.batch_size]
        batch_x1 = self.X_img[index * self.batch_size:(index + 1) * self.batch_size]
        batch_y = self.y[index * self.batch_size:(index + 1) * self.batch_size]

        X0_batch = np.asarray([self.__get_input1(que) for que in batch_x0])
        X1_batch = np.asarray([self.__get_input2(colab_path+path) for path in batch_x1])
        y_batch = np.asarray([self.__get_output(c) for c in batch_y])
       
        return tuple([X0_batch, X1_batch]), y_batch
    
    def __len__(self):
        return len(self.indexes) // self.batch_size

In [ ]:
import numpy as np

from keras.models import Sequential
from my_classes import DataGenerator

# Parameters
params = {'dim': (32,32,32),
          'batch_size': 64,
          'n_classes': 6,
          'n_channels': 1,
          'shuffle': True}

# Datasets
partition = # IDs
labels = # Labels

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)

# Design model
model = Sequential()
[...] # Architecture
model.compile()

# Train model on dataset
model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6)